In [2]:
# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
#Cat = melanoma y Dog = benign
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import numpy as np
import matplotlib.pyplot as plt

In [3]:
base_dir = os.getcwd()

# Directorios de clases (10 clases)
clases = ['piel-normal', 'lunar', 'melanoma', 'acne', 'carcinoma-de-celulas-escamosas', 'varicela', 
          'piel-quemada', 'queratosis-actinica', 'carcinoma-de-celulas-basales', 'queratosis-seborreica']

In [7]:
# Crear las carpetas de entrenamiento y validación para las 10 clases
try:
    os.mkdir(os.path.join(base_dir, 'diagnostico'))
    os.mkdir(os.path.join(base_dir, 'diagnostico', 'training'))
    os.mkdir(os.path.join(base_dir, 'diagnostico', 'testing'))

    for clase in clases:
        os.mkdir(os.path.join(base_dir, 'diagnostico', 'training', clase))
        os.mkdir(os.path.join(base_dir, 'diagnostico', 'testing', clase))

except OSError as e:
    print(f"An error occurred: {e}")

In [7]:
# Función para dividir los datos en entrenamiento y prueba
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = len(files) - training_length
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:]

    for filename in training_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING, filename)
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TESTING, filename)
        copyfile(this_file, destination)

In [9]:
# Divide los datos para las 10 clases
try:
    
    split_size = .95
    for clase in clases:
        source_dir = os.path.join(base_dir, "diagnostico-dataset", clase)
        training_dir = os.path.join(base_dir, "diagnostico", "training", clase)
        testing_dir = os.path.join(base_dir, "diagnostico", "testing", clase)
        split_data(source_dir, training_dir, testing_dir, split_size)
except OSError as e:
    print(f"An error occurred: {e}")

In [21]:
# Construcción del modelo actualizado
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),  # Nueva capa con más filtros
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 clases con softmax
])

# Compilación del modelo para múltiples clases
model.compile(optimizer=RMSprop(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Generadores de datos para entrenamiento y validación
TRAINING_DIR = os.path.join(base_dir, 'diagnostico', 'training')
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

VALIDATION_DIR = os.path.join(base_dir, 'diagnostico', 'testing')
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(150, 150))


Found 49172 images belonging to 10 classes.
Found 2594 images belonging to 10 classes.


In [23]:
# Entrenamiento del modelo
history = model.fit_generator(train_generator,
                              epochs=15,  # Aumentamos el número de épocas para mayor precisión
                              verbose=1,
                              validation_data=validation_generator)

Epoch 1/30
1537/1537 [==============================] - 2444s 2s/step - loss: 0.8825 - accuracy: 0.7085 - val_loss: 0.7625 - val_accuracy: 0.7390
Epoch 2/30
1537/1537 [==============================] - 1753s 1s/step - loss: 0.8646 - accuracy: 0.7133 - val_loss: 0.7569 - val_accuracy: 0.7402
Epoch 3/30
1537/1537 [==============================] - 1823s 1s/step - loss: 0.8502 - accuracy: 0.7179 - val_loss: 1.0971 - val_accuracy: 0.6241
Epoch 4/30
1537/1537 [==============================] - 1786s 1s/step - loss: 0.8401 - accuracy: 0.7211 - val_loss: 0.7941 - val_accuracy: 0.7344
Epoch 5/30
1537/1537 [==============================] - 1837s 1s/step - loss: 0.8295 - accuracy: 0.7231 - val_loss: 0.7446 - val_accuracy: 0.7440
Epoch 6/30
1537/1537 [==============================] - 1821s 1s/step - loss: 0.8198 - accuracy: 0.7254 - val_loss: 0.7476 - val_accuracy: 0.7444
Epoch 7/30
1537/1537 [==============================] - 1840s 1s/step - loss: 0.8092 - accuracy: 0.7286 - val_loss: 0.8340 -

In [43]:
# Guardar el modelo
save_dir = os.path.join(base_dir, 'models')
model.save(os.path.join(save_dir, 'clasificador_10_clases.h5'))

In [4]:

save_dir = os.path.join(base_dir, 'models')
model_path = os.path.join(save_dir, 'clasificador_10_clases.h5')
model = tf.keras.models.load_model(model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# Visualización de los resultados del entrenamiento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [9]:
# Predicción en nuevas imágenes
img_path = os.path.join(base_dir, "Varicela.jpg")
from tensorflow.keras.preprocessing import image

# Cargar la imagen y preprocesarla
img = image.load_img(img_path, target_size=(150, 150)) 
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0  

# Realizar la predicción
classes = model.predict(x)

# Obtener la clase con mayor probabilidad
predicted_class_idx = np.argmax(classes[0])
probabilidad = classes[0][predicted_class_idx]

# Asignar la clase correspondiente al índice
clase_predicha = clases[predicted_class_idx]

# Mostrar la clase predicha y la probabilidad
print(f"Predicción: {clase_predicha} (Probabilidad: {probabilidad*100:.2f}%)")


Predicción: queratosis-seborreica (Probabilidad: 39.58%)
